## Extract the simulation template data for execution
In this example we extract the afsim tooling template data.

1. Query and pull from the API the template model into memory.
2. Extract the meta data
3. Extract the input and output paramters and their mapping
4. Create an API call to the AFSIM server based on the template data
5. Execute the AFSIM model

In [1]:
# Will not run here due to privleges issue
# Just copy and paste (minus the '!') and run in a terminal with privledges
# !pip install --trusted-host nexus.mgmt.internal --index-url https://nexus.mgmt.internal:8443/repository/pypi-internal/simple pandas --trusted-host nexus.mgmt.internal --no-cache-dir

In [2]:
from __future__ import print_function

import requests
import pandas as pd
import json

# Assuming 'df' is your DataFrame
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', None)  # Show full width of showing columns
pd.set_option('display.width', None)  # Auto-detect the display width

# SysMLAPI Server
host = "http://sysmlapiserver:9000"
projectHumanMName = "studyMapper_AFSIM"

In [3]:
# Just all the projectId & commitId
map_projectId = "4116bf53-f169-468a-99f4-658922d8f7a9" # analysis_templates/studyMapper_AFSIM_A.ipynb
# map_commitId = "ba7988ee-0885-4c87-8970-bf7a130fae30"

In [ ]:
# Query the full model
query_input = {
  '@type': 'Query'
  }

payload = json.dumps(query_input)
query_url = f"{host}/projects/{map_projectId}/query-results"
query_response = requests.post(query_url, json=query_input)

if query_response.status_code == 200:
    query_response_json = query_response.json()
    formatted_json = json.dumps(query_response_json, indent=4)
print(formatted_json)

In [4]:
# Now lets loop though the model and extract all of the meta data
data = json.loads(formatted_json)

reference_usages = []

# Function to find a value for a given element ID
def find_value(element_id, json_data):
    for item in json_data:
        if item.get('@id') == element_id:
            if '@type' in item and item['@type'] == 'LiteralString':
                return item.get('value')
            if 'value' in item:
                return find_value(item['value']['@id'], json_data)
    return "Value not found or not a LiteralString"

# Process each item in the JSON data
for item in data:
    if item.get('@type') == 'ReferenceUsage':
        name = item.get('name', 'No name')
        ruId = item.get('@id', 'No ID')
        owned_elements = item.get('ownedElement', [])
        for oe in owned_elements:
            oe_id = oe.get('@id')
            oe_value = find_value(oe_id, data)
            reference_usages.append({'Name': name, 'uuid': ruId, 'Value': oe_value})

# Convert the list of dictionaries into a pandas DataFrame for easy table visualization
df_meta = pd.DataFrame(reference_usages)

# Display the DataFrame
df_meta

,Name,uuid,Value
0,p_cargoCapacity,3a6e0ef9-83cd-4340-8e93-d154f2e62e63,missionModel_simple::uav::cargoCap
1,c1,bd84ef15-ffd1-4aaa-b16b-f6b1e1978913,missionModel_simple::UAV_Performance::UAV_Speed::requiredSpeedConstarint
2,name,5065d38c-2591-4d38-9fb6-7aa9d6edbb96,a_cargoCap
3,projectId,8fdfb12f-7ea7-4ef4-97b4-252ac18cbc8b,5db467f8-02bf-43cd-98f4-4315f66ddb53
4,p_speed,4ac004e0-a0a3-457f-bc76-f58386407e4e,Value not found or not a LiteralString
5,p_speed,057d2f5b-fde0-4f14-8a1a-9661ef771c5b,missionModel_simple::uav::speed
6,p_range,5fb74a47-a827-4706-b450-8cb2aa037488,Value not found or not a LiteralString
7,commitId,e97fb036-fa90-4e52-9411-6fca2bcb2187,9164a608-d997-4fed-92a4-fb213c46d938
8,name,b904bf52-1d13-4875-95ca-cf71c5d6663c,a_range
9,c2,e57dd58d-8cbb-4e5e-a3a2-b6b87e5920fc,missionModel_simple::UAV_Performance::UAV_Range::requiredRangeConstarint


In [5]:
# Next we need to pull from the API the mapped items and map them so that we can:
# 1 - Build a call to the api from the template
# 2 - Create an input paramters file
# 3 - Call the simulation tool per the template and exicute it with the input file.

def find_item_by_id(data, item_id):
    """Utility function to find an item by its @id."""
    return next((item for item in data if item.get("@id") == item_id), None)

results = []

# Step 1: Iterate through all items to find initial ReferenceUsage instances
for item in data:
    if item.get("@type") == "ReferenceUsage":
        # Step 2: For each found ReferenceUsage, find linked MetadataUsage
        for owned_element in item.get("ownedElement", []):
            metadata_usage = find_item_by_id(data, owned_element.get("@id"))
            if metadata_usage and metadata_usage.get("@type") == "MetadataUsage":
                # Step 3: For each found MetadataUsage, find another ReferenceUsage
                for mu_owned_element in metadata_usage.get("ownedElement", []):
                    second_ref_usage = find_item_by_id(data, mu_owned_element.get("@id"))
                    if second_ref_usage and second_ref_usage.get("@type") == "ReferenceUsage":
                        # Step 4: For the second ReferenceUsage, find LiteralString
                        for sr_owned_element in second_ref_usage.get("ownedElement", []):
                            literal_string = find_item_by_id(data, sr_owned_element.get("@id"))
                            if literal_string and literal_string.get("@type") == "LiteralString":
                                results.append({
                                    "Model Paramter": item.get("name"),
                                    "Initial ReferenceUsage ID": item.get("@id"),
                                    "MetadataUsage ID": metadata_usage.get("@id"),
                                    "Second ReferenceUsage ID": second_ref_usage.get("@id"),
                                    "LiteralString ID": literal_string.get("@id"),
                                    "Simulation Value": literal_string.get("value")
                                })

# `results` will contain the list of dictionaries with the required information.
# Creating a pandas DataFrame
df_map = pd.DataFrame(results)

# Printing the DataFrame in a table format
df_map

,Model Paramter,Initial ReferenceUsage ID,MetadataUsage ID,Second ReferenceUsage ID,LiteralString ID,Simulation Value
0,p_speed,4ac004e0-a0a3-457f-bc76-f58386407e4e,016b3b5a-88ca-42c8-ac48-8e1ad33e6097,865ee3b9-c84e-4027-9259-b63eb485a148,ab5879e8-40fb-4a9e-aed0-e3bd80086338,speed
1,p_range,5fb74a47-a827-4706-b450-8cb2aa037488,18ea3c89-2b35-4888-95f9-874e8fe24f9e,3a1ef8c4-b216-4ba5-a0f3-85f36b4929d0,461d15cc-22cb-409b-9a25-60b9f8b8b1ea,range
2,c_calculatedRange,544dee61-b56d-47b6-a1cb-fe0fb76b1ffc,be18c601-ad68-4732-b59f-44471cf5969a,b904bf52-1d13-4875-95ca-cf71c5d6663c,82c36d04-4d8e-424b-9207-402080ecd8fb,a_range
3,p_cargoCapacity,fa879cd8-89ce-41c2-b159-cd002acc5254,1f02dde9-c2cc-4a4f-b955-d682a5838d77,e242a119-a608-4e29-9ff3-e1cb6eb5bd29,d062616c-4883-4dd8-9e26-9feee2b2731c,cargoCap
4,c_calculatedSpeed,6a13b21f-69fe-4682-80b9-057de2218352,afcde7db-ff6c-431d-a6c1-794a83a7b2c6,c86b55ce-4889-4e0e-ad5b-9bddd0910974,5439c55e-adaf-46ee-8e08-be5c69a6bbf7,a_speed
5,c_calculatedCargoCap,8c0bba27-3778-453f-9e4a-2fc074d24cae,a6c7a164-c7fe-4290-a464-bf40bd9ef56e,5065d38c-2591-4d38-9fb6-7aa9d6edbb96,ebab56a2-1fc9-435b-bf01-e439d8cf407d,a_cargoCap


In [11]:
# Now we can build and test a call to AFSIM :)
apiCall = df_meta[(df_meta['Name'] == 'apiCall')]['Value']
print(apiCall)

23    curl -X POST $uri -F 'file=@$manifestFile' -F 'start_file=$startFile'
Name: Value, dtype: object


In [13]:
# Extract API call variables from the datatable
api_uri = df_meta[df_meta["Name"].str.contains("uri", case=False)]["Value"].values[0]
apiCall_Value = df_meta[df_meta["Name"].str.contains("apiCall", case=False)]["Value"].values[0]
manifestFile_Value = df_meta[df_meta["Name"].str.contains("maifestFile", case=False)]["Value"].values[0]
startFile_Value = df_meta[df_meta["Name"].str.contains("startFile", case=False)]["Value"].values[0]
print(api_uri, apiCall_Value, manifestFile_Value, startFile_Value)

http://10.10.20.222:5000/afsim_execute curl -X POST $uri -F 'file=@$manifestFile' -F 'start_file=$startFile' /mnt/cdefs/templates/afsim/afsim_test.zip 1v1.afsim


In [14]:
# Test the AFSIM endpoint
# 1 - Update the call with the previous information from the template data retrieved from the api
files = {
    'file': (manifestFile_Value.split('/')[-1], open(manifestFile_Value, 'rb')),
    'start_file': (None, startFile_Value),
}

response = requests.post(api_uri, files=files)
print(response.text)


{
  "message": "Mission execution complete",
  "output_dir": "/mnt/cdefs/results/1v1"
}



In [11]:
# cURL example if ever needed
updated_apiCall_Value = apiCall_Value.replace('$manifestFile', manifestFile_Value).replace('$startFile', startFile_Value).replace('$', api_uri)
updated_apiCall_Value

"curl -X POST http://10.10.20.222:5000/afsim_executeuri -F 'file=@/mnt/cdefs/templates/afsim/afsim_test.zip' -F 'start_file=1v1.afsim'"

In [ ]:
# Now we need to add for each paramter get the value and then update the input file to the tool